In [2]:
# IMPORT ALL REQUIRED LIBRARIES
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig


from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.19.0


In [3]:
from azureml.core import Workspace, Experiment
ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')
experiment_name = 'udacity-capstone-project'
exp=Experiment(ws, experiment_name)
project_folder = './automlproject'
run = exp.start_logging()

Workspace name: quick-starts-ws-132062
Azure region: southcentralus
Subscription id: f5091c60-1c3c-430f-8d81-d802f6bf2414
Resource group: aml-quickstarts-132062


In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# Choose a name for your CPU cluster
from azureml.core.compute_target import ComputeTargetException
cpu_cluster_name = "cpu-cluster-mla"

   # Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                              max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

    cpu_cluster.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [5]:
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.dataset import Dataset
from azureml.data.datapath import DataPath

url_path = 'https://raw.githubusercontent.com/dib1979/Temporary/main/IRIS.csv'
dataset = TabularDatasetFactory.from_delimited_files(path=url_path)

dataset = dataset.register(workspace=ws,
                    name='iris_data',
                    description='Udacity Project')
dataset.to_pandas_dataframe().head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [8]:
df = dataset.to_pandas_dataframe()
df.describe()

,sepal_length,sepal_width,petal_length,petal_width
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667
std,0.828066,0.433594,1.764420,0.763161
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [9]:
df['species'].value_counts()

Iris-versicolor    50
Iris-setosa        50
Iris-virginica     50
Name: species, dtype: int64

In [10]:
X= df.drop(['species'], axis=1)

In [11]:
y = df['species']


In [12]:
from sklearn.model_selection import train_test_split
import pandas as pd

(X_train, X_test, y_train, y_test) = train_test_split(X, y, test_size= 0.3, random_state = 0)
label = 'species'

In [13]:
train_data_df = pd.concat([X_train, y_train], axis=1)

train_data_df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
60,5.0,2.0,3.5,1.0,Iris-versicolor
116,6.5,3.0,5.5,1.8,Iris-virginica
144,6.7,3.3,5.7,2.5,Iris-virginica
119,6.0,2.2,5.0,1.5,Iris-virginica
108,6.7,2.5,5.8,1.8,Iris-virginica


In [14]:
# save training data in tabular format to allow for remote run
if not os.path.isdir('data'):  # create data folder if it does not exist
    os.mkdir('data')
    
if not os.path.exists('project_folder'):  # create project folder if it does not exist
    os.makedirs('project_folder')

# Save the train data to a csv file to be uploaded to the datastore
pd.DataFrame(train_data_df).to_csv("data/train_data.csv", index=False)

# Upload the training data as a tabular dataset for access during training on remote compute
# upload to data store
ds = ws.get_default_datastore()
ds.upload(src_dir='./data', target_path='automlclassifier', overwrite=True, show_progress=True)

 # access datastore during training on remote compute
train_data = TabularDatasetFactory.from_delimited_files(path=ds.path('automlclassifier/train_data.csv'))

Uploading an estimated of 1 files
Uploading ./data/train_data.csv
Uploaded ./data/train_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


In [15]:
# Set parameters for AutoMLConfig

# define autoconfig settings
automl_settings = {
    "enable_early_stopping" : True,
    "iteration_timeout_minutes": 5,
    "max_concurrent_iterations": 4,
    "max_cores_per_iteration": -1,    
    "featurization": 'auto',
    "verbosity": logging.INFO,
}

# define automl autconfig parameters
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task= 'classification',
    primary_metric='accuracy',
    enable_onnx_compatible_models=True,
    compute_target=cpu_cluster, # included to allow for remote compute
    training_data= train_data,
    label_column_name= label,
    path = project_folder,
    n_cross_validations=3,
    debug_log = "automl_errors.log",    
    **automl_settings)

In [16]:
#Submit experiment for remote run
automl_run = exp.submit(automl_config, show_output = True)

Running on remote.
No run_configuration provided, running on cpu-cluster-mla with default configuration
Running on remote compute: cpu-cluster-mla
Parent Run ID: AutoML_92c4ac84-b98b-4fc3-91f0-6090eded1910

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Beginning model selection.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high

In [17]:
# show run details
from azureml.widgets import RunDetails
RunDetails(automl_run).show()

# wait for completion
automl_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more about high cardinality feature handling: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

****************************************************************************************************
ITERATION: The iteration being evaluated.
PIPELINE: A summary d

{'runId': 'AutoML_92c4ac84-b98b-4fc3-91f0-6090eded1910',
 'target': 'cpu-cluster-mla',
 'status': 'Completed',
 'startTimeUtc': '2020-12-27T19:28:36.064236Z',
 'endTimeUtc': '2020-12-27T20:04:26.549752Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'cpu-cluster-mla',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-capstone-project","subscription_id":"f5091c60-1c3c-430f-8d81-d802f6bf2414","resource_group":"aml-quickstarts-132062","workspace_name":"quick-starts-ws-132062","region":"southcentralus","compute_target":"cpu-cluster-mla","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":3,"y_min":null,"y_max":nul

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [18]:
best_automl, best_fit_model = automl_run.get_output()
print(best_fit_model)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                  min_samples_leaf=0.06157894736842105,
                                                                                                  min_samples_split=0.2442105263157895,
                                                                                                  min_weight_fraction_le

In [19]:
print(best_automl)

Run(Experiment: udacity-capstone-project,
Id: AutoML_92c4ac84-b98b-4fc3-91f0-6090eded1910_44,
Type: azureml.scriptrun,
Status: Completed)


In [23]:
#TODO: Save the best model

import joblib
joblib.dump(best_fit_model, 'best_fit_automl_model.pkl')

['best_fit_automl_model.pkl']

In [24]:
# Register the Model
from azureml.core.model import Model
model = Model.register(workspace = ws, model_name = 'best_fit_automl_model', model_path = 'best_fit_automl_model.pkl')
print(model.name, model.id, model.version, sep='\t')

Registering model best_fit_automl_model
best_fit_automl_model	best_fit_automl_model:1	1


In [25]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core import Environment
from azureml.core.model import Model

service_name = 'my-automl-service'
env = Environment.get(workspace=ws, name="AzureML-Tutorial")  
env.python.conda_dependencies.add_pip_package("scikit-learn")

inference_config = InferenceConfig(entry_script='score3.py', environment=env)
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=4, enable_app_insights=True)

model = Model(ws,name='best_fit_automl_model')
service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)

service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.........................................
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [26]:
print(service.state)
print(service.scoring_uri)
print(service.swagger_uri)

Healthy
http://66faec51-cade-4ca4-b7c6-2a0cbf4d8453.southcentralus.azurecontainer.io/score
http://66faec51-cade-4ca4-b7c6-2a0cbf4d8453.southcentralus.azurecontainer.io/swagger.json


In [27]:
import requests
import json

# URL for the web service
scoring_uri = 'http://66faec51-cade-4ca4-b7c6-2a0cbf4d8453.southcentralus.azurecontainer.io/score'

# Set the content type
headers = {'Content-Type': 'application/json'}

data = {"data":
        [
          {
            "sepal_length": 6.0,
            "sepal_width": 2.0,
            "petal_length": 5.6,
            "petal_width": 3.2,
           
          },
          {
            "sepal_length": 3.0,
            "sepal_width": 3.0,
            "petal_length": 1.6,
            "petal_width": 4.2,
          }
      ]
    }
# Convert to JSON string
input_data = json.dumps(data)

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)

print("Response Code : ", resp.status_code)
print("Predicted Value : ",resp.text)

Response Code :  200
Predicted Value :  ["Iris-virginica", "Iris-setosa"]


In [28]:
# Web Service Logs
print(service.get_logs())

2020-12-27T20:16:16,747322000+00:00 - iot-server/run 
2020-12-27T20:16:16,745760300+00:00 - gunicorn/run 
2020-12-27T20:16:16,786281700+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_a1736710baabf05d37337a101fa8d6d1/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_a1736710baabf05d37337a101fa8d6d1/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_a1736710baabf05d37337a101fa8d6d1/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_a1736710baabf05d37337a101fa8d6d1/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_a1736710baabf05d37337a101fa8d6d1/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2020-12-27T20:16:16,799475700+00:00 - rsyslog/run 
rsyslogd